In [24]:
vdw_type = "protor"
n = 4

T = 2.5
ε = 0.1
L = 5
rs = 1.4
η = 0.3665
bounds = 200.0
σ_t = 1.25
σ_r = 0.15
overlap_jump = 0.0
overlap_slope = 0.85
delaunay_eps = 10.0
comment = ""

energy_specifier = "oj_$(replace(string(overlap_jump), "." => "_"))_os_$(replace(string(overlap_slope), "." => "_"))"
simulation_time_minutes = 12.0 * 60.0

720.0

In [27]:
Ts = [1.4, 1.6, 1.8, 2.0, 2.2, 2.4, 2.6, 2.8, 3.0]
simulations_per_combination = 5
mol_type = "6sag"
open("configs/$(vdw_type)_$(energy_specifier)_config.txt", "w") do io
    println(io,"ArrayTaskID input_string")
    for (i, T) in enumerate(Ts)
        template_file = "assets/input/jld2/2_6sag_protor_hardsphere_assembly.jld2"
        output_directory = "assets/output/assembly/$(mol_type)_$(vdw_type)_$(energy_specifier)/"
        for j in 1:simulations_per_combination
            name = "$((i-1)*simulations_per_combination+j)_$(mol_type)_$(vdw_type)_$(energy_specifier)"
            input_string = escape_string("name=\"$name\";T=$(T);rs=$rs;η=$η;L=$L;ε=$ε;σ_t=$σ_t;σ_r=$σ_r;overlap_jump=$overlap_jump;overlap_slope=$overlap_slope;simulation_time_minutes=$simulation_time_minutes;output_directory=\"$output_directory\";template_file=\"$template_file\";delaunay_eps=$delaunay_eps;comment=\"$comment\"")
            println(io, "$((i-1)*simulations_per_combination + j) $input_string")
        end
    end
end

In [28]:
hours = Int(round(simulation_time_minutes / 60.0))
days = hours ÷ 24
remaining_hours = hours % 24
remaining_hours_string = remaining_hours < 10 ? "0$(remaining_hours)" : string(remaining_hours)
buffer_time_string = simulation_time_minutes < 5 ? "0$(Int(simulation_time_minutes)+2)" : "30"

open("$(vdw_type)_$(energy_specifier)_script.job", "w") do io
    println(io, "#!/bin/bash")
    println(io, "#SBATCH --job-name=SolvationSimulations")
    println(io, "#SBATCH --time=0$(days)-$(remaining_hours_string):$(buffer_time_string)")
    println(io, "#SBATCH --ntasks=1")
    println(io, "#SBATCH --cpus-per-task=1")
    println(io, "#SBATCH --array=1-$(simulations_per_combination * length(Ts))")
    println(io, "#SBATCH --chdir=/work/spirandelli/MorphoMolMonteCarlo/")
    println(io, "#SBATCH -o ./tools/hpc/%a.out # STDOUT")
    println(io, "")
    println(io, "export http_proxy=http://proxy2.uni-potsdam.de:3128 #Setting proxy, due to lack of Internet on compute nodes.")
    println(io, "export https_proxy=http://proxy2.uni-potsdam.de:3128")
    println(io, "")
    println(io, "module purge")
    println(io, "module load lang/Julia/1.7.3-linux-x86_64")
    println(io, "")
    println(io, "# Specify the path to the config file")
    println(io, "config=tools/hpc/configs/$(vdw_type)_$(energy_specifier)_config.txt")
    println(io, "")
    println(io, "# Extract the variables from config file")
    println(io, "config_string=\$(awk -v ArrayTaskID=\$SLURM_ARRAY_TASK_ID '\$1==ArrayTaskID {print \$2}' \$config)")
    println(io, "")
    println(io, "julia -e \"$(escape_string("include(\"hpc/hmc_call.jl\"); hmc_call(\"\$config_string\")"))\"")
    println(io, "")
    println(io, "# copy back results")
    println(io, "mkdir -p ~/outputs/ && cp -r assets/output/assembly/* ~/outputs/")
end